In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
os.getcwd()

'C:\\Users\\tanub\\Desktop\\midterm'

In [3]:
nextgenraw = pd.read_csv("NextGenRaw.txt", sep="\t")
nextgenraw

,GENE,Control1,Control2,Nitrate1,Nitrate2
0,AT5G16970,7,17,4,5
1,AT2G43120,36,31,32,15
2,AT3G18710,16,23,37,15
3,AT4G25880,58,78,59,56
4,AT1G78930,3,3,3,1
...,...,...,...,...,...
25625,AT4G13615,392,385,535,304
25626,AT1G24405,1,2,3,1
25627,AT2G27490,22,19,18,16
25628,AT1G71740,8,4,3,2


In [9]:
biogrid = pd.read_csv("AthBIOGRID.txt", sep="\t")
biogrid

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,251838,828230,832208,13519,17483,AT4G00020,AT5G20850,BRCA2(IV),RAD51,BRCA2A|BREAST CANCER 2 like 2A|EDA20|EMBRYO SA...,...,15014444,3702,3702,Low Throughput,-,-,-,-,-,BIOGRID
1,251839,828230,821860,13519,7192,AT4G00020,AT3G22880,BRCA2(IV),DMC1,BRCA2A|BREAST CANCER 2 like 2A|EDA20|EMBRYO SA...,...,15014444,3702,3702,Low Throughput,-,-,-,-,-,BIOGRID
2,265014,836259,818903,21503,4240,AT5G61380,AT2G43010,TOC1,PIF4,APRR1|AtTOC1|MFB13.13|MFB13_13|PRR1|PSEUDO-RES...,...,14634162,3702,3702,Low Throughput,-,-,-,-,-,BIOGRID
3,265015,836259,825075,21503,10390,AT5G61380,AT3G59060,TOC1,PIL6,APRR1|AtTOC1|MFB13.13|MFB13_13|PRR1|PSEUDO-RES...,...,14634162,3702,3702,Low Throughput,-,-,-,-,-,BIOGRID
4,265016,836259,836259,21503,21503,AT5G61380,AT5G61380,TOC1,TOC1,APRR1|AtTOC1|MFB13.13|MFB13_13|PRR1|PSEUDO-RES...,...,14634162,3702,3702,Low Throughput,-,-,-,-,-,BIOGRID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56193,2618335,838388,827028,23627,12325,AT1G18080,AT4G13940,ATARCA,MEE58,RACK1A|RACK1A_AT|RECEPTOR FOR ACTIVATED C KINA...,...,31606202,3702,3702,High Throughput,-,-,-,BiFC assay,-,BIOGRID
56194,2618336,838388,821921,23627,7253,AT1G18080,AT3G23400,ATARCA,FIB4,RACK1A|RACK1A_AT|RECEPTOR FOR ACTIVATED C KINA...,...,31606202,3702,3702,High Throughput,-,-,-,BiFC assay,-,BIOGRID
56195,2618337,838388,828889,23627,14176,AT1G18080,AT4G27750,ATARCA,ISI1,RACK1A|RACK1A_AT|RECEPTOR FOR ACTIVATED C KINA...,...,31606202,3702,3702,High Throughput,-,-,-,BiFC assay,-,BIOGRID
56196,2618338,838388,836652,23627,21894,AT1G18080,AT5G65270,ATARCA,RABA4a,RACK1A|RACK1A_AT|RECEPTOR FOR ACTIVATED C KINA...,...,31606202,3702,3702,High Throughput,-,-,-,BiFC assay,-,BIOGRID


In [4]:
con = sqlite3.connect("td2201_midterm.sqlite")
c = con.cursor()

In [11]:
biogrid.to_sql("biogrid", con, index=False, if_exists="append")

C:\Users\tanub\miniconda3\lib\site-packages\pandas\core\generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [5]:
nextgenraw.to_sql("nextgenraw", con, index=False, if_exists="append")

In [42]:
pd.read_sql("""
            SELECT * FROM sqlite_master
            WHERE type='table';
""", con)

,type,name,tbl_name,rootpage,sql
0,table,DataType,DataType,2,CREATE TABLE DataType(DTID INT PRIMARY KEY NOT...
1,table,Data,Data,4,CREATE TABLE Data(DID INT PRIMARY KEY NOT NULL...
2,table,biomart,biomart,283,"CREATE TABLE ""biomart"" (\n""Gene stable ID"" TEX..."
3,table,DataAttributes,DataAttributes,8357,CREATE TABLE DataAttributes(DAID INT PRIMARY K...
4,table,nextgenraw,nextgenraw,9378,"CREATE TABLE ""nextgenraw"" (\n""GENE"" TEXT,\n ""..."
5,table,biogrid,biogrid,8362,"CREATE TABLE ""biogrid"" (\n""#BioGRID Interactio..."
6,table,Relationship,Relationship,14080,CREATE TABLE Relationship(RID INT PRIMARY KEY ...


In [6]:
go2gene = pd.read_sql("""
            SELECT distinct(c.DID) as DID_Data_1, b.DID as DID_Data_2
            from biomart a join data b join data c 
            on a."Gene stable ID"=b.DataName and a."GO term accession"=c.DataName;

""", con)
go2gene

,DID_Data_1,DID_Data_2
0,28662,1985
1,28663,1985
2,28664,1985
3,28665,1985
4,28666,1985
...,...,...
154288,30931,3647
154289,28838,4613
154290,28706,4613
154291,28678,4613


In [7]:
go2gene.insert(0, 'Relationship', 'GOTerm2Gene')
go2gene

,Relationship,DID_Data_1,DID_Data_2
0,GOTerm2Gene,28662,1985
1,GOTerm2Gene,28663,1985
2,GOTerm2Gene,28664,1985
3,GOTerm2Gene,28665,1985
4,GOTerm2Gene,28666,1985
...,...,...,...
154288,GOTerm2Gene,30931,3647
154289,GOTerm2Gene,28838,4613
154290,GOTerm2Gene,28706,4613
154291,GOTerm2Gene,28678,4613


In [8]:
go2gene.insert(3, 'RelValue', '')
go2gene #This is the df for go term to gene relationship of relationship table

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,GOTerm2Gene,28662,1985,
1,GOTerm2Gene,28663,1985,
2,GOTerm2Gene,28664,1985,
3,GOTerm2Gene,28665,1985,
4,GOTerm2Gene,28666,1985,
...,...,...,...,...
154288,GOTerm2Gene,30931,3647,
154289,GOTerm2Gene,28838,4613,
154290,GOTerm2Gene,28706,4613,
154291,GOTerm2Gene,28678,4613,


In [17]:
interaction = pd.read_sql("""
                select distinct(a."Experimental System") as Relationship,
                b.DID as DID_Data_1, c.DID as DID_Data_2, a."Experimental System Type" as RelValue 
                from biogrid a join Data b join Data c on 
                a."Systematic Name Interactor A"=b.DataName and a."Systematic Name Interactor B"=c.DataName;
""", con)
interaction

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Two-hybrid,1,789,physical
1,Two-hybrid,1,790,physical
2,Two-hybrid,2,89,physical
3,Two-hybrid,2,21,physical
4,Two-hybrid,2,2,physical
...,...,...,...,...
54551,FRET,1352,1334,physical
54552,FRET,1352,7347,physical
54553,FRET,1352,10471,physical
54554,FRET,1352,10487,physical


In [21]:
control1 = pd.read_sql("""
            select distinct(b.DID) as DID_Data_2, a.Control1 as RelValue
            from nextgenraw a join Data b 
            on a.GENE=b.DataName
""", con)
control1 #df for experiment 1

,DID_Data_2,RelValue
0,6823,7
1,4303,36
2,1985,16
3,10489,58
4,5906,3
...,...,...
25625,18996,392
25626,27576,1
25627,27577,22
25628,27578,8


In [24]:
control1.insert(0, 'Relationship', 'Experiment2Gene')
control1.insert(1, 'DID_Data_1', 34808)
control1 #complete experiment1 df

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34808,6823,7
1,Experiment2Gene,34808,4303,36
2,Experiment2Gene,34808,1985,16
3,Experiment2Gene,34808,10489,58
4,Experiment2Gene,34808,5906,3
...,...,...,...,...
25625,Experiment2Gene,34808,18996,392
25626,Experiment2Gene,34808,27576,1
25627,Experiment2Gene,34808,27577,22
25628,Experiment2Gene,34808,27578,8


In [25]:
control2 = pd.read_sql("""
            select distinct(b.DID) as DID_Data_2, a.Control2 as RelValue
            from nextgenraw a join Data b 
            on a.GENE=b.DataName
""", con)
control2 #df for experiment 2

,DID_Data_2,RelValue
0,6823,17
1,4303,31
2,1985,23
3,10489,78
4,5906,3
...,...,...
25625,18996,385
25626,27576,2
25627,27577,19
25628,27578,4


In [26]:
control2.insert(0, 'Relationship', 'Experiment2Gene')
control2.insert(1, 'DID_Data_1', 34809)
control2 #complete experiment2 df

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34809,6823,17
1,Experiment2Gene,34809,4303,31
2,Experiment2Gene,34809,1985,23
3,Experiment2Gene,34809,10489,78
4,Experiment2Gene,34809,5906,3
...,...,...,...,...
25625,Experiment2Gene,34809,18996,385
25626,Experiment2Gene,34809,27576,2
25627,Experiment2Gene,34809,27577,19
25628,Experiment2Gene,34809,27578,4


In [27]:
nitrate1 = pd.read_sql("""
            select distinct(b.DID) as DID_Data_2, a.Nitrate1 as RelValue
            from nextgenraw a join Data b 
            on a.GENE=b.DataName
""", con)
nitrate1 #df for experiment 3

,DID_Data_2,RelValue
0,6823,4
1,4303,32
2,1985,37
3,10489,59
4,5906,3
...,...,...
25625,18996,535
25626,27576,3
25627,27577,18
25628,27578,3


In [28]:
nitrate1.insert(0, 'Relationship', 'Experiment2Gene')
nitrate1.insert(1, 'DID_Data_1', 34810)
nitrate1 #complete experiment3 df

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34810,6823,4
1,Experiment2Gene,34810,4303,32
2,Experiment2Gene,34810,1985,37
3,Experiment2Gene,34810,10489,59
4,Experiment2Gene,34810,5906,3
...,...,...,...,...
25625,Experiment2Gene,34810,18996,535
25626,Experiment2Gene,34810,27576,3
25627,Experiment2Gene,34810,27577,18
25628,Experiment2Gene,34810,27578,3


In [29]:
nitrate2 = pd.read_sql("""
            select distinct(b.DID) as DID_Data_2, a.Nitrate2 as RelValue
            from nextgenraw a join Data b 
            on a.GENE=b.DataName
""", con)
nitrate2 #df for experiment 4

,DID_Data_2,RelValue
0,6823,5
1,4303,15
2,1985,15
3,10489,56
4,5906,1
...,...,...
25625,18996,304
25626,27576,1
25627,27577,16
25628,27578,2


In [30]:
nitrate2.insert(0, 'Relationship', 'Experiment2Gene')
nitrate2.insert(1, 'DID_Data_1', 34811)
nitrate2 #complete experiment4 df

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34811,6823,5
1,Experiment2Gene,34811,4303,15
2,Experiment2Gene,34811,1985,15
3,Experiment2Gene,34811,10489,56
4,Experiment2Gene,34811,5906,1
...,...,...,...,...
25625,Experiment2Gene,34811,18996,304
25626,Experiment2Gene,34811,27576,1
25627,Experiment2Gene,34811,27577,16
25628,Experiment2Gene,34811,27578,2


In [31]:
x1 = control1.append(control2, ignore_index=True)
x1

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34808,6823,7
1,Experiment2Gene,34808,4303,36
2,Experiment2Gene,34808,1985,16
3,Experiment2Gene,34808,10489,58
4,Experiment2Gene,34808,5906,3
...,...,...,...,...
51255,Experiment2Gene,34809,18996,385
51256,Experiment2Gene,34809,27576,2
51257,Experiment2Gene,34809,27577,19
51258,Experiment2Gene,34809,27578,4


In [32]:
x2 = x1.append(nitrate1, ignore_index=True)
x2

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34808,6823,7
1,Experiment2Gene,34808,4303,36
2,Experiment2Gene,34808,1985,16
3,Experiment2Gene,34808,10489,58
4,Experiment2Gene,34808,5906,3
...,...,...,...,...
76885,Experiment2Gene,34810,18996,535
76886,Experiment2Gene,34810,27576,3
76887,Experiment2Gene,34810,27577,18
76888,Experiment2Gene,34810,27578,3


In [33]:
x3 = x2.append(nitrate2, ignore_index=True)
x3 #the whole experiment df

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,Experiment2Gene,34808,6823,7
1,Experiment2Gene,34808,4303,36
2,Experiment2Gene,34808,1985,16
3,Experiment2Gene,34808,10489,58
4,Experiment2Gene,34808,5906,3
...,...,...,...,...
102515,Experiment2Gene,34811,18996,304
102516,Experiment2Gene,34811,27576,1
102517,Experiment2Gene,34811,27577,16
102518,Experiment2Gene,34811,27578,2


In [36]:
relationship = go2gene.append(x3, ignore_index=True)
relationship #go term2gene + experiment2gene

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,GOTerm2Gene,28662,1985,
1,GOTerm2Gene,28663,1985,
2,GOTerm2Gene,28664,1985,
3,GOTerm2Gene,28665,1985,
4,GOTerm2Gene,28666,1985,
...,...,...,...,...
256808,Experiment2Gene,34811,18996,304
256809,Experiment2Gene,34811,27576,1
256810,Experiment2Gene,34811,27577,16
256811,Experiment2Gene,34811,27578,2


In [37]:
relationship1 = relationship.append(interaction, ignore_index=True)
relationship1

,Relationship,DID_Data_1,DID_Data_2,RelValue
0,GOTerm2Gene,28662,1985,
1,GOTerm2Gene,28663,1985,
2,GOTerm2Gene,28664,1985,
3,GOTerm2Gene,28665,1985,
4,GOTerm2Gene,28666,1985,
...,...,...,...,...
311364,FRET,1352,1334,physical
311365,FRET,1352,7347,physical
311366,FRET,1352,10471,physical
311367,FRET,1352,10487,physical


In [39]:
relationship1.insert(0, 'RID', range(1, 1+len(relationship1)))
relationship1 #my relationship table df

,RID,Relationship,DID_Data_1,DID_Data_2,RelValue
0,1,GOTerm2Gene,28662,1985,
1,2,GOTerm2Gene,28663,1985,
2,3,GOTerm2Gene,28664,1985,
3,4,GOTerm2Gene,28665,1985,
4,5,GOTerm2Gene,28666,1985,
...,...,...,...,...,...
311364,311365,FRET,1352,1334,physical
311365,311366,FRET,1352,7347,physical
311366,311367,FRET,1352,10471,physical
311367,311368,FRET,1352,10487,physical


In [41]:
c.execute("""
            CREATE TABLE Relationship(RID INT PRIMARY KEY NOT NULL,
            Relationship TEXT, DID_Data_1 INT, DID_Data_2 INT,
            RelValue VARCHAR,
            FOREIGN KEY(DID_Data_1) REFERENCES Data(DID),
            FOREIGN KEY(DID_Data_2) REFERENCES Data(DID))
""")

In [43]:
relationship1.to_sql("Relationship", con, index=False, if_exists="append")

In [44]:
pd.read_sql("""
            select * from Relationship
""", con)

,RID,Relationship,DID_Data_1,DID_Data_2,RelValue
0,1,GOTerm2Gene,28662,1985,
1,2,GOTerm2Gene,28663,1985,
2,3,GOTerm2Gene,28664,1985,
3,4,GOTerm2Gene,28665,1985,
4,5,GOTerm2Gene,28666,1985,
...,...,...,...,...,...
311364,311365,FRET,1352,1334,physical
311365,311366,FRET,1352,7347,physical
311366,311367,FRET,1352,10471,physical
311367,311368,FRET,1352,10487,physical


In [49]:
pd.read_sql("""
            SELECT * FROM sqlite_master
            WHERE type='table';
""", con)

,type,name,tbl_name,rootpage,sql
0,table,DataType,DataType,2,CREATE TABLE DataType(DTID INT PRIMARY KEY NOT...
1,table,Data,Data,4,CREATE TABLE Data(DID INT PRIMARY KEY NOT NULL...
2,table,DataAttributes,DataAttributes,8357,CREATE TABLE DataAttributes(DAID INT PRIMARY K...
3,table,Relationship,Relationship,14080,CREATE TABLE Relationship(RID INT PRIMARY KEY ...


In [46]:
c.execute("""
        DROP TABLE biomart
""")

In [47]:
c.execute("""
        DROP TABLE biogrid
""")

In [48]:
c.execute("""
        DROP TABLE nextgenraw
""")

In [50]:
con.close()